### Imports and model specifications

In [1]:
# Import dependencies
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import grid as gr

# We use the grid version of syft with some extension methods
from grid import syft as sy

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
james = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")

alice.connect()
bob.connect()
james.connect()
dan.connect()

gr.connect_all_nodes([bob, alice, james, dan])

In [2]:
data_shape = (1, 1)
data = th.zeros(data_shape)

In [6]:
# Model Owner
# Support fetch plan + AST tensor
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__(id="encrypted-linear-model-2")
        self.fc1 = nn.Linear(1,1)
        self.add_to_state(["fc1"])

    def forward(self, x):
        return self.fc1(x)

plan = Net()
    
plan.build(data)
print(plan(data))

tensor([[0.8032]], grad_fn=<AddmmBackward>)


In [7]:
plan.encrypt(bob, james, crypto_provider=dan)

<Net Net id:encrypted-linear-model-2 owner:me built>

In [8]:
alice.serve_encrypted_model(plan)

True

In [9]:
alice.models

['encrypted-linear-model', 'encrypted-linear-model-2']

In [10]:
encrypted_model = alice.download_model("encrypted-linear-model-2")

In [11]:
x_sh = data.encrypt(bob, james, crypto_provider=dan)

In [13]:
encrypted_model(x_sh).request_decryption()

tensor([[0.8030]])